In [1]:
import instructor, llm_validator
from openai import OpenAI
from pydantic import BaseModel, ValidationError, BeforeValidator
from typing import List
from typing_extensions import Annotated


In [2]:
# Enables `response_model`
client = instructor.patch(OpenAI())

## Get structured LLM output

In [3]:
class ArticleDetail(BaseModel):
    summary: str
    headline: str
    entities: List[str]

In [4]:
article_example = """
Manchester United announced Sunday that British petrochemical billionaire Sir Jim Ratcliffe purchased a 25% stake in the famed English Premier League soccer club.

The sale was confirmed by Manchester United and Ratcliffe-controlled INEOS, a London-based multinational conglomerate.

INEOS said in a statement that Ratcliffe will acquire 25% of the club’s Class B shares and up to 25% of its publicly available Class A shares “and provide an additional $300 million intended to enable future investment into Old Trafford,” the club’s historic stadium. The Glazer family, which owns the majority of the club, and Class A shareholders will receive the same price of $33.00 per share, the club said. On Friday, shares of Manchester United (MANU) closed at $19.81.

In May, Forbes valued Manchester United at $6 billion, just behind Real Madrid as the world’s most valuable soccer club. Shares of Manchester United had been down nearly 13% in 2023.

The deal still requires approval from the English Premier League.

“We are delighted to have agreed (to) this deal with Sir Jim Ratcliffe and INEOS,” said Avram Glazer and Joel Glazer, Manchester United’s executive co-chairmen and directors, in a statement. “As part of the strategic review we announced in November 2022, we committed to look at a variety of alternatives to help enhance Manchester United, with a focus on delivering success for our men’s, women’s and Academy teams.”

That statement added that “the joint ambition is to create a world-class football operation building on the Club’s many existing strengths, including the successful off-pitch performance that it continues to enjoy.”

The sale comes after over a year of rumors that the Florida-based Glazer family, which has owned the club since 2005 and also owns the NFL’s Tampa Bay Buccaneers, was interested in accepting offers. There have been multiple rounds of bids, which had included Qatari investor Sheikh Jassim bin Hamad Al Thani, who withdrew his bid in October, according to multiple reports.

Manchester United has struggled in recent years under the Glazer family’s ownership. The club, which have had four full-time managers since long-time boss Sir Alex Ferguson stepped down in 2013, have not won a UEFA Champions League title since 2008, and have not captured a Premier League title since 2013.

Manchester United are currently eighth in the Premier League table.

Ratcliffe, 71, was born in Failsworth, Lancashire, which is part of Greater Manchester. He has described himself as “a local boy and a lifelong supporter” of Manchester United. Ratcliffe, one of Britain’s richest men with a net worth of $19.1 billion, according to Bloomberg’s billionaire index, previously launched a failed bid for London club Chelsea, which fetched $5.3 billion in May 2022.

Ratcliffe also owns soccer clubs OGC Nice in France and FC Lausanne-Sport in Switzerland.
"""

In [6]:
prompt = f"""
You are an experienced news editor. 
Your job is to summarize an article, give it an enticing headline, and label the relevant figures, places or entities in the article.
The provided article is copied below:
###
{article_example}
###
"""

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=ArticleDetail,
    messages=[
        {"role": "user", "content": prompt},
    ],
    max_tokens=300
)

In [10]:
response.summary

"British billionaire Sir Jim Ratcliffe has purchased a 25% stake in Manchester United, the famed English Premier League soccer club. Ratcliffe, who is worth $19.1 billion, will acquire shares and provide $300 million for future investment into the club's stadium, Old Trafford. The sale, which still requires approval from the English Premier League, comes after rumors of the Glazer family's interest in accepting offers. Manchester United has struggled under the Glazer family's ownership and has not won a Champions League title since 2008 or a Premier League title since 2013."

In [11]:
response.headline

'Billionaire Sir Jim Ratcliffe Buys 25% Stake in Manchester United'

In [12]:
response.entities

['Sir Jim Ratcliffe',
 'Manchester United',
 'English Premier League',
 'INEOS',
 'Old Trafford',
 'Glazer family',
 'Champions League']

## Use LLM to validate output content

In [16]:
class CarDetail(BaseModel):
    model: str
    price: str
    description: Annotated[
        str,
        BeforeValidator(llm_validator("Make sure the description discusses a car model", openai_client=client))
    ]

In [19]:
try:
    qa = CarDetail(
        model="Aurora GT8",
        price="$95,000 - $120,000",
        description="The Aurora GT8 is a sleek, high-performance sports car known for its revolutionary electric powertrain that delivers outstanding acceleration and range. With a cutting-edge design featuring aerodynamic curves and an ultra-light carbon fiber body, the GT8 combines luxury with eco-friendly efficiency. Its interior boasts state-of-the-art technology, including a fully digital dashboard and advanced driver-assist features."
    )
except ValidationError as e:
    print(e)


In [20]:
try:
    qa = CarDetail(
        model="Tesla Model X",
        price="$90,000 - $120,000",
        description="This ride is notable for being the tallest hybrid roller coaster in North America and the fastest and steepest hybrid coaster in the world. Iron Gwazi combines a traditional wooden structure with modern steel track, delivering an incredibly smooth yet intense experience."
    )
except ValidationError as e:
    print(e)


1 validation error for CarDetail
description
  Assertion failed, The description does not discuss a car model. [type=assertion_error, input_value='This ride is notable for...yet intense experience.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/assertion_error
